# SEP532 인공지능 이론과 실제
## Deep Learning Practice 
#### Prof. Ho-Jin Choi
#### School of Computing, KAIST

---

## Text generation with an RNN 
In this practice, we will learn how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

- The model is **character-based**. When training started, the model did not know how to spell an English word, or that words were even a unit of text.
- The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.
- As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

#### Import libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

#### Download the Shakespeare dataset
Change the following line to run this code on your own data.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)

/root/.keras/datasets/shakespeare.txt


#### Read the data
First, look in the text:

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


#### Vectorize the text
Before training, we need to **map strings to a numerical representation**. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [7]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))

It converts form tokens to character IDs, padding with `0`:

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[41, 42, 43, 44, 45, 46, 47], [64, 65, 66]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the padding and `[UNK]` tokens are set the same way.

In [9]:
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

#### The prediction task
Given a character, or a sequence of characters, **what is the most probable next character?** This is the task we're training the model to perform. The **input to the model will be a sequence of characters**, and we train the model to **predict the output—the following character at each time step**.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

#### Create training examples and targets
Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length + 1`. For example, say s`eq_length` is 4 and our text is "Hello". **The input sequence would be "Hell", and the target sequence "ello"**.

To do this first use the [`tf.data.Dataset.from_tensor_slices`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#from_tensor_slices) function to convert the text vector into a stream of character indices.
- `tf.data.Dataset.from_tensor_slices`: Creates a Dataset whose elements are slices of the given tensors.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([20, 49, 58, ..., 47, 10,  2])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [17]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [18]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [19]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [20]:
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### Create training batches
We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to **shuffle the data and pack it into batches**.

- [`tf.data.Dataset.shuffle`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#shuffle)(buffer_size, seed=None, reshuffle_each_iteration=None)
    - Randomly shuffles the elements of this dataset.
- [`tf.data.Dataset.batch`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#batch)(batch_size, drop_remainder=False)
    - Combines consecutive elements of this dataset into batches.

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Build The Model
Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

- `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
- [`tf.keras.layers.GRU`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/GRU): A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)
<img src=https://miro.medium.com/max/2400/1*dhq14CzJijlqjf7IlDB0uw.png>

 
- `tf.keras.layers.Dense`: The output layer, with vocab_size outputs.

In [39]:
# Length of the vocabulary in chars
# vocab_size = len(vocab)
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])

In [40]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

<img src=https://www.tensorflow.org/tutorials/text/images/text_generation_training.png>

#### Try the model
Now run the model to see that it behaves as expected.

First check the shape of the output:

In [41]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") 

(64, 100, 67) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           17152     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 67)            68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

_Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop._

Try it for the first example in the batch:

In [43]:
# 1D vector (100, 1)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices.shape)

(100, 1)


This gives us, at each timestep, a prediction of the next character index:
- [`tf.squeeze`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/squeeze): Removes dimensions of size 1 from the shape of a tensor.

In [44]:
# (100,)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([29, 32,  5, 16, 10, 15, 25, 28,  0, 25, 62, 65, 36, 20, 41, 23, 52,
       21, 65, 39, 21, 25, 47, 17, 53, 55, 37, 13,  2, 40, 32, 10, 21, 15,
       14, 46,  4, 58, 55, 65, 63,  7, 64, 19, 27,  6, 11, 31, 30,  2, 35,
       11, 53, 30, 52,  1, 20, 27, 20,  2, 22, 28, 57, 40, 52, 45, 23, 65,
       16,  6,  5,  8, 59, 12, 19, 45, 45, 32, 24, 41, 20, 27, 28, 10, 20,
       24, 52, 14, 23,  5, 20, 64, 34, 22, 56, 32, 30, 16, 13, 12])

Decode these to see the text predicted by this **untrained model**:

In [45]:
print('Input:\n', text_from_ids(input_example_batch[0]).numpy())
print()
print('Next Char Predictions:\n', text_from_ids(sampled_indices).numpy())

Input:
 b'thy tongue?\n\nMessenger:\nAh, one that was a woful looker-on\nWhen as the noble Duke of York was slain,'

Next Char Predictions:
 b"OR$B.AKNKvyVFaIlGyYGKgCmoW;\nZR.GA?f!royw'xEM&3QP\nU3mPl[UNK]FMF\nHNqZleIyB&$,s:EeeRJaFMN.FJl?I$FxTHpRPB;:"


#### Train the model
At this point the problem can be treated as a standard classification problem. **Given the previous RNN state, and the input this time step, predict the class of the next character.**

##### Attach an optimizer, and a loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.

In [46]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [47]:
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print(f'Prediction shape: {example_batch_predictions.shape}   # (batch_size, sequence_length, vocab_size)')
print(f'Mean loss:        {mean_loss}')

Prediction shape: (64, 100, 67)   # (batch_size, sequence_length, vocab_size)
Mean loss:        4.204878330230713


Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [48]:
model.compile(optimizer='adam', loss=loss)

#### Configure checkpoints
Use a [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to ensure that checkpoints are saved during training:

In [49]:
# Directory where the checkpoints will be saved
checkpoint_dir = './shakespeare'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Execute the training
To keep training time reasonable, use 20 epochs to train the model.

In [50]:
history = model.fit(dataset, 
                    epochs=20, 
                    callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 6s 26ms/step - loss: 3.2996
Epoch 2/20
172/172 [==============================] - 5s 26ms/step - loss: 2.0550
Epoch 3/20
172/172 [==============================] - 5s 25ms/step - loss: 1.7472
Epoch 4/20
172/172 [==============================] - 5s 26ms/step - loss: 1.5724
Epoch 5/20
172/172 [==============================] - 5s 25ms/step - loss: 1.4672
Epoch 6/20
172/172 [==============================] - 5s 26ms/step - loss: 1.4032
Epoch 7/20
172/172 [==============================] - 5s 27ms/step - loss: 1.3510
Epoch 8/20
172/172 [==============================] - 5s 26ms/step - loss: 1.3114
Epoch 9/20
172/172 [==============================] - 5s 27ms/step - loss: 1.2681
Epoch 10/20
172/172 [==============================] - 5s 27ms/step - loss: 1.2376
Epoch 11/20
172/172 [==============================] - 5s 26ms/step - loss: 1.2018
Epoch 12/20
172/172 [==============================] - 5s 27ms/step - loss: 1.1705
Epoch 13/20
1

### Generate Text

First, restore the latest checkpoint. Then, to keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

**To run the model with a different** `batch_size`, we need to **rebuild the model and restore the weights from the checkpoint**.

In [51]:
tf.train.latest_checkpoint(checkpoint_dir)

'./shakespeare/ckpt_20'

In [52]:
# Rebuild the model by changing batch size (=1) to predict new text
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# Load the weight of the model we trained 
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# Change the batch size from 64 to 1
model.build(tf.TensorShape([1, None]))

In [53]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            17152     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 67)             68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


#### The prediction loop

The following code block generates the text:

* It starts by choosing a **start string**, initializing the RNN state and setting the number of characters to generate.
* Get the **prediction distribution of the next character using the start string** and the RNN state.
* Use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.
* The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize and make paragraphs, and it imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [54]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [ids_from_chars(s).numpy() for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0]

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(chars_from_ids(predicted_id))

    return start_string + tf.strings.join(text_generated)

In [55]:
print(generate_text(model, start_string=u"ROMEO: ").numpy().decode())

ROMEO: Montague: I am late.

MICHA:
But you are that succession bless your fire,
I'll tell him where my father's Sentinance may yet be free
As doom this place, that sensill prove meet and neighbours' law.
You fight, adieu: gentlemen! go to; we must half war with
these quiet that he is as estad queen in all this ill;
And all this idll points of Volsces? Grumio
Take me old means and fruit--ischerns, but most
indeed!
No do I thought you'll say ayes
Before I part them over well, when I should please
I carry together, Edward at Pursuith thou art swift'st confine into
Fristing will never reach him here.
Perhup to heaven, my life.

CAPULET:
My lord, I hold me from false eyes
With humours will also turn his judge, vast what George?
Sim did thee ro, it sins and long auss of him.

ANGELO:
No, madam; there's in ears there,
Thou madnstrone widow of his cornt:
say, to prove a sold mother, or those means;
There stands with strongs are fullous loss.

GONZALO:
And then I fear me.
Tou are salce in, Lad

The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.